In [2]:
import numpy as np
import sys
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [89]:
# ALL OF THE FUNCTIONS WE USE

# FOR THE FIRST PROBLEM

def G1(x1, x2):
    return np.array([x1**2 - 1, x2**2 - 1])

def F1(x1, x2):
    return np.array([x1**2 - x2**2 + 3, x1*x2 + 6])

def H1(G, H, x1, x2, t):
    return t*G(x1, x2) + (1 - t)*F(x1, x2)

def testJac1(x1, x2):
    return np.array([[2*x1, -2*x2],[x2, x1]])

def Jac_h(x1, x2, t):
    return np.array([[2*x1, -2*x2*(1 - t)], [2*x2*t + x1*(1 - t), x2*(1 - t)]])


##def Jacobian(x1, x2, t):
##    return np.array([[t*2*x1 + (1 - t)*2*x1, (1 - t)*(-2*x2)], [(1 - t)*x2, t*2*x2 + (1 - t)*x1]])



In [84]:
def single_Homotop2(F, G, JacF, JacH, px1, px2, stepsize):
    
    # FIND THE FIRST VALUE SO WE CAN START APPENDING OUR ARRAY OF TUPLES
    
    # OUR STARTING SOLUTIONS TO G(x1, x2)
    initial_tuple = (px1, px2)
    x1_curr = initial_tuple[0]
    x2_curr = initial_tuple[1]
    
    # OUR JACOBIAN of F
    jac_currF = JacF(x1_curr, x2_curr) 
    
    # THE FIRST NEWTON ITERATION
    second_tuple = initial_tuple - np.matmul(Invert(jac_currF), F(x1_curr, x2_curr))
    
    # ADD THIS TO OUR ARRAY OF TOUPLES
    vals = np.array([initial_tuple, second_tuple])
    
    # OUR NUMBER OF STEPS OVER 1 TIME
    n = int(1/stepsize)
    
    # PATH TRACKING
    
    for i in range(1, n):

        # OUR TIME AT THIS ITERATION
        current_time = 1 - i*stepsize

        #OUR VALUES AT THIS TIME
        val_at_curr = vals[-1]
        x1_curr = val_at_curr[0]
        x2_curr = val_at_curr[1]

        #OUR JACOBIANS AT THIS TIME
        jac_currH = JacH(x1_curr, x2_curr, current_time) #, 1 - stepsize*i)
        jac_currF = JacF(x1_curr, x2_curr)

        # VALUES WE GET FOR G AND F
        G_tuple_curr = G(x1_curr, x2_curr)
        F_tuple_curr = F(x1_curr, x2_curr)

        # PREDICTOR:
        H_derriv_t = G_tuple_curr - F_tuple_curr
        x_prime_predict = val_at_curr - np.matmul(Invert(jac_currH), H_derriv_t)

        # OUR CORRECTING
        Newton_corrector = np.matmul(Invert(jac_currF), F(x1_curr, x2_curr)) 
        corrected_delta =  Newton_corrector - x_prime_predict

        # FINAL ODE ITERATION
        new_vals = val_at_curr + corrected_delta*stepsize
        
        vals = np.vstack((vals, new_vals))
        
    return vals
    



In [85]:
# OUR PATH TRACKING
#       path_tracked_curr = current_time*G_tuple_curr + (1 - current_time)*F_tuple_curr

## TESTING 
ogsol1 = [1, 1]
ogsol2 = [1, -1]
ogsol3 = [-1, -1]
ogsol4 = [-1, 1]

newsol1 = single_Homotop2(F1, G1, testJac1, Jac_h, 1, 1, 0.5)[-1]
newsol2 = single_Homotop2(F1, G1, testJac1, Jac_h, 1, -1, 0.1)[-1]
newsol3 = single_Homotop2(F1, G1, testJac1, Jac_h, -1, -1, 0.1)[-1]
newsol4 = single_Homotop2(F1, G1, testJac1, Jac_h, -1, 1, 0.1)[-1]


## check
print(f"QUESTION2:")
print(f"OUR SOLUTIONS RELATED WITH THE STARTER {ogsol1} FOR F YIELDS {newsol1} (resulting in {F1(newsol1[0], newsol1[1])} error)")
print(f"OUR SOLUTIONS RELATED WITH THE STARTER {ogsol2} FOR F YIELDS {newsol2} (resulting in {F1(newsol2[0], newsol2[1])} error)")
print(f"OUR SOLUTIONS RELATED WITH THE STARTER {ogsol3} FOR F YIELDS {newsol3} (resulting in {F1(newsol3[0], newsol3[1])} error)")
print(f"OUR SOLUTIONS RELATED WITH THE STARTER {ogsol4} FOR F YIELDS {newsol4} (resulting in {F1(newsol4[0], newsol4[1])} error)")

QUESTION2:
OUR SOLUTIONS RELATED WITH THE STARTER [1, 1] FOR F YIELDS [-2.23992773  0.62989163] (resulting in [7.62051278 4.58908828] error)
OUR SOLUTIONS RELATED WITH THE STARTER [1, -1] FOR F YIELDS [1.28206516 1.87026854] (resulting in [1.14578668 8.39780613] error)
OUR SOLUTIONS RELATED WITH THE STARTER [-1, -1] FOR F YIELDS [1.99699605 1.18503058] (resulting in [5.58369576 8.36650138] error)
OUR SOLUTIONS RELATED WITH THE STARTER [-1, 1] FOR F YIELDS [-1.28206516 -1.87026854] (resulting in [1.14578668 8.39780613] error)


In [90]:
#FOR THE SECOND PROBLEM

def G2(x1, x2, x3):
    return np.array([x1**2 - 1, x2**2 - 1, x3**2 - 1])

def F2(x1, x2):
    return np.array([x1 - 2*x2 + x2**2 + x2**3 - 4, x1 + x2 - 2*x2**2 + 2, 0])

def testJac2(x1, x2):
    return np.array([[1, -2 + 2*x2 + 3*x2**2, 0], [1, 1 - 4*x2, 0], [0, 0, 0]])

def Jac_h2(x1, x2, x3, t):
    return np.array([[2*x1*t + (1 - t)*(1), (1-t)*(-2 + 2*x2 + 3*x2*2), 0],[(1 - t), t*2*x2 + (1 - t)* (1 - 4*x2), 0], [0, 0, t*2*x3]])

def Invert(A):
    return LA.inv(A)

def pseudoInv(A):
    return LA.pinv(A)

In [93]:
## FOR DEGREE 3

def single_Homotop3(F, G, JacF, JacH, px1, px2, px3, stepsize):
    
    # FIND THE FIRST VALUE SO WE CAN START APPENDING OUR ARRAY OF TUPLES
    
    # OUR STARTING SOLUTIONS TO G(x1, x2)
    initial_tuple = (px1, px2, px3)
    x1_curr = initial_tuple[0]
    x2_curr = initial_tuple[1]
    x2_curr = initial_tuple[2]
    
    # OUR JACOBIAN 
    jac_currf = JacF(x1_curr, x2_curr) 
    
    # THE FIRST NEWTON ITERATION
    second_tuple = initial_tuple - np.matmul(pseudoInv(jac_currf), F(x1_curr, x2_curr))
    
    # ADD THIS TO OUR ARRAY OF TOUPLES
    vals = np.array([initial_tuple, second_tuple])
    
    n = int(1/stepsize)
    
    # PATH TRACKING
    
    for i in range(1, n):

        # OUR TIME AT THIS ITERATION
        current_time = 1 - i*stepsize

        #OUR VALUES AT THIS TIME
        val_at_curr = vals[-1]
        x1_curr = val_at_curr[0]
        x2_curr = val_at_curr[1]
        x3_curr = val_at_curr[2]

        #OUR JACOBIANS AT THIS TIME
        jac_currH = JacH(x1_curr, x2_curr, x3_curr, current_time) #, 1 - stepsize*i)
        jac_currF = JacF(x1_curr, x2_curr)

        # VALUES WE GET FOR G AND F
        G_tuple_curr = G(x1_curr, x2_curr, x3_curr)
        F_tuple_curr = F(x1_curr, x2_curr)

        # PREDICTOR:
        H_derriv_t = G_tuple_curr - F_tuple_curr
        x_prime_predict = val_at_curr - np.matmul(pseudoInv(jac_currH), H_derriv_t)

        # OUR CORRECTING
        Newton_corrector = np.matmul(pseudoInv(jac_currF), F(x1_curr, x2_curr)) 
        corrected_delta =  Newton_corrector - x_prime_predict

        # FINAL ODE ITERATION
        new_vals = val_at_curr + corrected_delta*stepsize
        
        vals = np.vstack((vals, new_vals))
        
    return vals

In [95]:


##print([x1g, x2g])
##print([x1_g, x2_g])
##F2(x1_g, x2_g)

OGsol1 = [1, 1, 1]
OGsol2 = [-1, -1, -1]
OGsol3 = [1, 1, -1]
OGsol4 = [1, -1, 1]
OGsol5 = [-1, 1, 1]
OGsol6 = [1, -1, -1]
OGsol7 = [-1, 1, -1]
OGsol8 = [-1, 1, 1]

NEWsol1 = single_Homotop3(F2, G2, testJac2, Jac_h2, 1, 1, 1, 0.1)[-1]
NEWsol2 = single_Homotop3(F2, G2, testJac2, Jac_h2, -1, -1, -1, 0.1)[-1]
NEWsol3 = single_Homotop3(F2, G2, testJac2, Jac_h2, 1, 1, -1, 0.1)[-1]
NEWsol4 = single_Homotop3(F2, G2, testJac2, Jac_h2, 1, -1, 1, 0.1)[-1]
NEWsol5 = single_Homotop3(F2, G2, testJac2, Jac_h2, -1, 1, 1, 0.1)[-1]
NEWsol6 = single_Homotop3(F2, G2, testJac2, Jac_h2, 1, -1, -1, 0.1)[-1]
NEWsol7 = single_Homotop3(F2, G2, testJac2, Jac_h2, -1, 1, -1, 0.1)[-1]
NEWsol8 = single_Homotop3(F2, G2, testJac2, Jac_h2, -1, 1, 1, 0.1)[-1]

print(f"QUESTION 3")
print(f"OUR SOLUTIONS RELATED TO {OGsol1} FOR F YIELDS {NEWsol1} (resulting in {F2(NEWsol1[0], NEWsol1[1])} ERROR)")
print(f"OUR SOLUTIONS RELATED TO {OGsol2} FOR F YIELDS {NEWsol2} (resulting in {F2(NEWsol2[0], NEWsol2[1])} ERROR)")
print(f"OUR SOLUTIONS RELATED TO {OGsol3} FOR F YIELDS {NEWsol3} (resulting in {F2(NEWsol3[0], NEWsol3[1])} ERROR)")
print(f"OUR SOLUTIONS RELATED TO {OGsol4} FOR F YIELDS {NEWsol4} (resulting in {F2(NEWsol4[0], NEWsol4[1])} ERROR)")
print(f"OUR SOLUTIONS RELATED TO {OGsol5} FOR F YIELDS {NEWsol5} (resulting in {F2(NEWsol5[0], NEWsol5[1])} ERROR)")
print(f"OUR SOLUTIONS RELATED TO {OGsol6} FOR F YIELDS {NEWsol6} (resulting in {F2(NEWsol6[0], NEWsol6[1])} ERROR)")
print(f"OUR SOLUTIONS RELATED TO {OGsol7} FOR F YIELDS {NEWsol7} (resulting in {F2(NEWsol7[0], NEWsol7[1])} ERROR)")
print(f"OUR SOLUTIONS RELATED TO {OGsol8} FOR F YIELDS {NEWsol8} (resulting in {F2(NEWsol8[0], NEWsol8[1])} ERROR)")


QUESTION 3
OUR SOLUTIONS RELATED TO [1, 1, 1] FOR F YIELDS [21.0252141  -2.8111606  -6.26901731] (resulting in [8.33461427 4.4088057  0.        ] ERROR)
OUR SOLUTIONS RELATED TO [-1, -1, -1] FOR F YIELDS [ 0.12140981 -0.53478985  6.26901731] (resulting in [-2.67596031  1.0146196   0.        ] ERROR)
OUR SOLUTIONS RELATED TO [1, 1, -1] FOR F YIELDS [ 2.03171707 -0.8693739   6.26901731] (resulting in [-0.13080649  1.65072122  0.        ] ERROR)
OUR SOLUTIONS RELATED TO [1, -1, 1] FOR F YIELDS [21.57326451  2.97712148 -6.26901731] (resulting in [46.86925275  8.82388135  0.        ] ERROR)
OUR SOLUTIONS RELATED TO [-1, 1, 1] FOR F YIELDS [21.0252141  -2.8111606  -6.26901731] (resulting in [8.33461427 4.4088057  0.        ] ERROR)
OUR SOLUTIONS RELATED TO [1, -1, -1] FOR F YIELDS [ 0.12140981 -0.53478985  6.26901731] (resulting in [-2.67596031  1.0146196   0.        ] ERROR)
OUR SOLUTIONS RELATED TO [-1, 1, -1] FOR F YIELDS [ 2.03171707 -0.8693739   6.26901731] (resulting in [-0.13080649  1